## Pruning

In [ ]:
!pip install tensorflow tensorflow-model-optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 3.6 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Epoch 1/5
1688/1688 [==============================] - 42s 24ms/step - loss: 0.1550 - accuracy: 0.9543 - val_loss: 0.0720 - val_accuracy: 0.9788
Epoch 2/5
1688/1688 [==============================] - 38s 23ms/step - loss: 0.0527 - accuracy: 0.9835 - val_loss: 0.0538 - val_accuracy: 0.9862
Epoch 3/5
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0342 - accuracy: 0.9898 - val_loss: 0.0468 - val_accuracy: 0.9885
Epoch 4/5
1688/1688 [==============================] - 37s 22ms/step - loss: 0.0225 - accuracy: 0.9931 - val_loss: 0.0504 - val_accuracy: 0.9877
Epoch 5/5
1688/1688 [==============================] - 38s 22ms/step - loss: 0.0149 - accuracy: 0.9951 - val_loss: 0.0501 - val_accuracy: 0.9878


In [ ]:
_, accuracy = model.evaluate(x_test, y_test)
print(f"Baseline test accuracy: {accuracy*100:.2f}%")


313/313 [==============================] - 2s 7ms/step - loss: 0.0411 - accuracy: 0.9870
Baseline test accuracy: 98.70%


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 5408)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               692352    
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 693962 (2.65 MB)
Trainable params: 693962 (2.65 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [ ]:
# Apply Pruning
pruning_schedule = tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                        final_sparsity=0.80,
                                                        begin_step=0,
                                                        end_step=np.ceil(x_train.shape[0] / 32).astype(np.int32) * 10)
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
model_for_pruning = prune_low_magnitude(model, pruning_schedule=pruning_schedule)

model_for_pruning.compile(optimizer='adam',
                          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                          metrics=['accuracy'])


In [ ]:
_, pruned_accuracy = model_for_pruning.evaluate(x_test, y_test)
print(f"Test accuracy after pruning: {pruned_accuracy*100:.2f}%")


313/313 [==============================] - 3s 8ms/step - loss: 0.0411 - accuracy: 0.9870
Test accuracy after pruning: 98.70%


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 5408)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               692352    
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 693962 (2.65 MB)
Trainable params: 693962 (2.65 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [ ]:
model_for_pruning.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d  (None, 26, 26, 32)        610       
 _1 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_max_po  (None, 13, 13, 32)        1         
 oling2d_1 (PruneLowMagnitu                                      
 de)                                                             
                                                                 
 prune_low_magnitude_flatte  (None, 5408)              1         
 n_1 (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_dense_  (None, 128)               1384578   
 2 (PruneLowMagnitude)                                           
                                                      

## Quantization

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10)
    ])
model.compile(optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Epoch 1/5
1688/1688 [==============================] - 67s 39ms/step - loss: 0.1652 - accuracy: 0.9504 - val_loss: 0.0724 - val_accuracy: 0.9797
Epoch 2/5
1688/1688 [==============================] - 38s 22ms/step - loss: 0.0541 - accuracy: 0.9827 - val_loss: 0.0541 - val_accuracy: 0.9838
Epoch 3/5
1688/1688 [==============================] - 36s 21ms/step - loss: 0.0355 - accuracy: 0.9888 - val_loss: 0.0495 - val_accuracy: 0.9865
Epoch 4/5
1688/1688 [==============================] - 39s 23ms/step - loss: 0.0228 - accuracy: 0.9931 - val_loss: 0.0539 - val_accuracy: 0.9870
Epoch 5/5
1688/1688 [==============================] - 37s 22ms/step - loss: 0.0155 - accuracy: 0.9952 - val_loss: 0.0542 - val_accuracy: 0.9858


In [ ]:
_, baseline_accuracy = model.evaluate(x_test, y_test)
print(f"Baseline test accuracy: {baseline_accuracy*100:.2f}%")


313/313 [==============================] - 2s 7ms/step - loss: 0.0500 - accuracy: 0.9847
Baseline test accuracy: 98.47%


In [ ]:
# Quantize the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_model = converter.convert()


In [ ]:
quantized_model.summary()

AttributeError: 'bytes' object has no attribute 'summary'

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 128)               692352    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 693962 (2.65 MB)
Trainable params: 693962 (2.65 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [ ]:
interpreter = tf.lite.Interpreter(model_content=quantized_model)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

# Run predictions on every image in the "test" dataset.
prediction_digits = []
for test_image in x_test:
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    interpreter.invoke()
    # Post-processing: remove batch dimension and find the digit with highest probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

# Compare prediction results with ground truth labels to calculate accuracy.
accurate_count = 0
for index in range(len(prediction_digits)):
    if prediction_digits[index] == y_test[index]:
        accurate_count += 1
accuracy = accurate_count * 1.0 / len(prediction_digits)

print(f'Test accuracy after quantization: {accuracy*100:.2f}%')


AttributeError: 'Interpreter' object has no attribute 'summary'

## Transfer Learning

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical


In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

170498071/170498071 [==============================] - 2s 0us/step


In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the base model
base_model.trainable = False


9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')  # CIFAR-10 has 10 classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test))

_, accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {accuracy*100:.2f}%")

Epoch 1/3
1563/1563 [==============================] - 62s 35ms/step - loss: 1.9298 - accuracy: 0.3013 - val_loss: 1.8687 - val_accuracy: 0.3188
Epoch 2/3
1563/1563 [==============================] - 53s 34ms/step - loss: 1.8302 - accuracy: 0.3355 - val_loss: 1.8324 - val_accuracy: 0.3394
Epoch 3/3
313/313 [==============================] - 9s 28ms/step - loss: 1.8128 - accuracy: 0.3438
Test accuracy: 34.38%
